# Rebel Leaders
Acosta, Benjamin, Reyko Huang, and Daniel Silverman. "Introducing ROLE: A Database of Rebel Leader Attributes in Armed Conflict." Journal of Peace Research. Online First.


What drives foreign state support for rebel organizations? While scholars have examined the geopolitical and organizational factors that fuel foreign support, the role of rebel leaders in this process remains under-studied. In this article, we propose that rebel leaders’ personal backgrounds shapes their ability to obtain foreign support during conflict. In particular, we argue that rebel leaders with significant prior international experiences – including study abroad, work abroad, military training abroad, and exile – are at an advantage in securing wartime external support for their organizations. These experiences provide opportunities for would-be rebel leaders to interact with a multitude of foreign individuals who may later enter politics or otherwise gain prominence in their respective societies, allowing them to build interpersonal social networks across borders. Such networks offer key points of contact when rebel leaders later seek foreign backing. We test this theory using data from the new Rebel Organization Leaders (ROLE) database, finding robust support for our argument as well as the broader role of rebel leader attributes in explaining external support. Our results underscore the value of incorporating individual leaders and their social networks more squarely into the study of modern war.

[Data Dictionary](https://www.rebelleaders.org/download)

In [1]:
import pandas as pd 
import numpy as np
import csv
import json

In [2]:
original_df = pd.read_csv('./rebel_leaders_org.csv')

In [3]:
original_df = original_df.iloc[:,1:]

In [4]:
original_df.head(3)

,revmodid,revmodcode,organization,english_translation,popular_alternative_name_or_acronym_most_recent,born,died,dead,age,duration,...,participates_in_negotiations,sas,sa,splinter_organization,ethnic,religious_sectarian,national,ideological,apocalyptic,fights_foreign_force
0,2,2B,Sendero Luminoso,Shining Path,NaN,1964,2012.0,1,48.0,48.0,...,0,0,0,1,0,0,0,1,0,0
1,3,3C,Frente Farabundo Martí para la Liberación Naci...,Farabundo Marti National Liberation Front/Unif...,FMLN,1979,NaN,0,35.0,15.0,...,1,0,0,0,0,0,0,1,0,1
2,4,4D,Óghlaigh na hÉireann/Cumann na mBan/Fianna na ...,Provisional Irish Republican Army (PIRA)/Irish...,PIRA,1969,2005.0,1,36.0,28.0,...,1,0,0,0,0,1,0,0,0,0


In [5]:
original_df.shape

(536, 91)

In [6]:
original_df.columns

Index(['revmodid', 'revmodcode', 'organization', 'english_translation',
       'popular_alternative_name_or_acronym_most_recent', 'born', 'died',
       'dead', 'age', 'duration', 'outcome_goal', 'adversary', 'achieve',
       'partialsuccess', 'completesuccess', 'orderedsuccess', 'polcommand',
       'nopolorgplus01', 'polorgplus', 'leader_s', 'leadershipdecapitation',
       'killed', 'arrested', 'executed', 'separate', 'overthrow_takeover',
       'empire', 'counterrev', 'reform', 'eliminate_expel', 'nsd', 'islamist',
       'leftist', 'right', 'anarchist', 'identity_group_representation',
       'excludedconstituentsatinception', 'size_year_one', 'size_peak',
       'hegemonic', 'unifiedfront', 'area_of_operation', 'adversary.1',
       'violent', 'nv', 'mixedapproach', 'as', 'ks', 'lethal', 'demonl',
       'demonc', 'turnedviolent', 'ties', 'tiefights', 'sponsors',
       'sponsorfights', 'west', 'latinamerica', 'mideastnafrica', 'africa',
       'centraleurasia', 'southasia', 's

### Born, Died, Duration of Rebel Organizations

In [7]:
original_df['born'].value_counts()

1989    23
2004    18
2011    17
2006    16
1982    14
        ..
1932     1
1924     1
1890     1
1951     1
2014     1
Name: born, Length: 79, dtype: int64

In [8]:
original_df['born'].isna().sum() 

0

In [9]:
original_df['died'].value_counts()

1989.0    18
2005.0    17
2012.0    14
2006.0    14
2009.0    13
2000.0    12
1983.0    11
1999.0    11
1995.0    11
2011.0    11
2013.0    11
1986.0    10
1990.0     9
1985.0     9
2007.0     9
2002.0     8
1994.0     8
1987.0     8
1992.0     7
1993.0     7
1991.0     7
2001.0     6
1988.0     6
2004.0     6
1982.0     6
1998.0     5
1984.0     5
1981.0     5
1997.0     5
2003.0     5
2010.0     4
2008.0     4
1996.0     4
1945.0     3
2014.0     3
1944.0     3
1979.0     2
1975.0     2
1973.0     2
1956.0     2
1969.0     2
1980.0     2
1960.0     2
1974.0     2
1963.0     1
1961.0     1
1953.0     1
1970.0     1
1968.0     1
1971.0     1
1950.0     1
1966.0     1
1958.0     1
1976.0     1
Name: died, dtype: int64

In [10]:
original_df['died'] = original_df['died'].fillna(0)

In [11]:
original_df['died'] = original_df['died'].astype(int)

In [12]:
original_df['duration'] = original_df['duration'].astype(int)

In [13]:
original_df['duration'].max()

148

In [14]:
original_df['duration'].min()

0

In [15]:
original_df['duration'].mean()

14.373134328358208

In [16]:
original_df['duration'].median()

9.0

##  Area of operation / Adversary

In [17]:
original_df['area_of_operation'].value_counts()

Lebanon                                         36
India                                           34
Iraq                                            19
Greece                                          16
Pakistan                                        13
                                                ..
Middle East; Tunisia                             1
Lebanon; Syria; disputed territories; Europe     1
Levant; Afghanistan                              1
Iraq; Iran; Turkey                               1
Turkey/Middle East                               1
Name: area_of_operation, Length: 213, dtype: int64

In [18]:
original_df['area_of_operation'].isnull().sum()

0

In [49]:
# going to clean up records with multiple countries 
original_df['area_of_operation'] = original_df['area_of_operation'].str.replace(';',',')
original_df['area_of_operation'] = original_df['area_of_operation'].str.upper()


In [50]:
# displays all value counts 
pd.set_option("display.max_rows", None)

In [51]:
original_df['area_of_operation'].value_counts()

LEBANON                                                      36
INDIA                                                        34
IRAQ                                                         19
GREECE                                                       16
PAKISTAN                                                     13
NEPAL                                                        13
UNITED STATES                                                10
PHILIPPINES                                                   8
COLOMBIA                                                      8
THAILAND                                                      8
FRANCE                                                        8
PAKISTAN, AFGHANISTAN                                         7
TURKEY                                                        7
CHILE                                                         6
BURMA                                                         6
EL SALVADOR                             

In [37]:
original_df.groupby('area_of_operation')['duration'].value_counts().to_frame()

duration
area_of_operation                                  duration          
Afghanistan                                        23               1
                                                   37               1
Afghanistan, Pakistan                              41               1
Albania                                            0                1
Algeria                                            6                1
                                                   13               1
                                                   18               1
Angola                                             12               1
                                                   36               1
                                                   48               1
Argentina                                          4                1
                                                   6                1
Argentina                                          0                1
Argentina, Italy, Bolivia                          21               1
Armenia, Lebanon, Nagorno-Karabakh, Europe, Nor... 42               1
Balochistan                                        7                1
Bangladesh                                         0                1
                                                   3                1
                                                   16               1
                                                   23               1
Belarus                                            0                2
Belgium                                            2                1
                                                   4                1
Benin                                              0                1
Bolivia                                            2                2
Brazil                                             6                1
Bulgaria                                           0                1
Burma                                              0                1
                                                   23               1
                                                   55               1
                                                   57               1
                                                   67               1
                                                   75               1
Burma, Thailand                                    26               1
Burundi                                            0                1
Cambodia                                           48               1
Cameroon, Nigeria                                  5                1
Central African Republic                           6                1
Central Asia                                       12               1
Central Asia, Pakistan                             1                1
                                                   16               1
Central Asia, Russia                               7                1
Chile                                              6                1
                                                   8                1
                                                   12               1
                                                   13               1
                                                   21               1
                                                   29               1
Chile, Europe                                      12               1
China                                              0                1
                                                   3                1
Colombia                                           1                1
                                                   4                1
                                                   8                1
                                                   10               1
                                                   15               1
 

In [45]:
# Aversary country/countries
original_df['adversary'] = original_df['adversary'].str.replace(';',',')
original_df['adversary'] = original_df['adversary'].str.upper()
original_df['adversary'].value_counts()

INDIA                                                                                            29
U.S.                                                                                             10
PAKISTAN                                                                                          9
NEPAL                                                                                             9
ISRAEL                                                                                            9
FRANCE                                                                                            8
UK                                                                                                8
THAILAND                                                                                          7
PHILIPPINES                                                                                       7
MYANMAR                                                                                           7


## Outcome Goals 

In [53]:
original_df['outcome_goal'] = original_df['outcome_goal'].str.upper()


original_df['outcome_goal'].value_counts()

TOPPLE THE INTERNATIONAL SYSTEM                                                                                                                     11
REESTABLISH THE ISLAMIC CALIPHATE                                                                                                                    9
ESTABLISH ISLAMIST STATE IN IRAQ                                                                                                                     7
EXPEL SYRIAN FORCES FROM LEBANON                                                                                                                     6
PREVENT REPUBLICAN SUCCESSION                                                                                                                        5
ESTABLISH INDEPENDENT TERAI STATE                                                                                                                    4
REPLACE ISRAEL WITH ISLAMIST STATE                                                            

## Leaders

In [54]:
original_df['leader_s'] = original_df['leader_s'].str.replace(';',',')
original_df['leader_s'] = original_df['leader_s'].str.replace('[leaderless]','Leaderless', regex=False)
original_df['leader_s'] = original_df['leader_s'].str.replace('[no identifiable leader]','No Identifiable Leader', regex=False)
original_df['leader_s'] = original_df['leader_s'].str.replace('[committee]','Committee', regex=False)
original_df['leader_s'] = original_df['leader_s'].str.replace('[','', regex=False)
original_df['leader_s'] = original_df['leader_s'].str.replace(']','', regex=False)
original_df['leader_s'] = original_df['leader_s'].str.upper()


In [55]:
original_df['leader_s'].value_counts()

LEADERLESS                                                                                                                                                                                                                                                                                                                                                         38
NO IDENTIFIABLE LEADER                                                                                                                                                                                                                                                                                                                                             38
COMMITTEE                                                                                                                                                                                                                                                                                   

## GDP 

In [58]:
original_df.groupby('area_of_operation')['gdp'].value_counts().to_frame()

gdp
area_of_operation                                  gdp         
AFGHANISTAN                                        47153.0    2
AFGHANISTAN, PAKISTAN                              688.0      1
ALBANIA                                            686.0      1
ALGERIA                                            2852.0     1
                                                   4967.0     1
                                                   6037.0     1
ANGOLA                                             768.0      1
                                                   840.0      1
                                                   4667.0     1
ARGENTINA                                          3532.0     1
                                                   8200.0     1
ARGENTINA                                          42569.0    1
ARGENTINA, ITALY, BOLIVIA                          9349.0     1
ARMENIA, LEBANON, NAGORNO-KARABAKH, EUROPE, NOR... 1574.0     1
BALOCHISTAN                                        1189.0     1
BANGLADESH                                         0.0        1
                                                   284.0      1
                                                   958.0      1
                                                   1189.0     1
BELARUS                                            1650.0     1
                                                   3849.0     1
BELGIUM                                            6084.0     1
                                                   12022.0    1
BENIN                                              392.0      1
BOLIVIA                                            968.0      1
                                                   23038.0    1
BRAZIL                                             1637.0     1
BULGARIA                                           2450.0     1
BURMA                                              0.0        3
                                                   359.0      1
                                                   1107.0     1
                                                   3104.0     1
BURMA, THAILAND                                    0.0        1
BURUNDI                                            271.0      1
CAMBODIA                                           308.0      1
CAMEROON, NIGERIA                                  1265.0     1
CENTRAL AFRICAN REPUBLIC                           457.0      1
CENTRAL ASIA                                       53042.0    1
CENTRAL ASIA, PAKISTAN                             1275.0     1
                                                   9132.0     1
CENTRAL ASIA, RUSSIA                               11700.0    1
CHILE                                              3891.0     2
                                                   2187.0     1
                                                   3883.0     1
                                                   5580.0     1
                                                   14394.0    1
CHILE, EUROPE                                      2877.0     1
CHINA                                              185.0      1
                                                   307.0      1
COLOMBIA                                           0.0        3
                                                   6240.0     3
                                                   1303.0     1
                                                   3405.0     1
COLOMBIA, LATIN AMERICA                            1387.0     1
COLUMBIA                                           0.0        1
CUBA                                               6051.0     1
CYPRUS                                             0.0        1
CZECHOSLOVAKIA                                     1852.0     1
                                                   5202.0     1
DENMARK                                            5543.0     1
DISPUTED TERRITORIES                               31282.0    2
                            

## Categories Exploration 

In [59]:
original_df[original_df['leftist'] == 1]

,revmodid,revmodcode,organization,english_translation,popular_alternative_name_or_acronym_most_recent,born,died,dead,age,duration,...,participates_in_negotiations,sas,sa,splinter_organization,ethnic,religious_sectarian,national,ideological,apocalyptic,fights_foreign_force
0,2,2B,Sendero Luminoso,Shining Path,NaN,1964,2012,1,48.0,48,...,0,0,0,1,0,0,0,1,0,0
1,3,3C,Frente Farabundo Martí para la Liberación Naci...,Farabundo Marti National Liberation Front/Unif...,FMLN,1979,0,0,35.0,15,...,1,0,0,0,0,0,0,1,0,1
2,4,4D,Óghlaigh na hÉireann/Cumann na mBan/Fianna na ...,Provisional Irish Republican Army (PIRA)/Irish...,PIRA,1969,2005,1,36.0,28,...,1,0,0,0,0,1,0,0,0,0
3,5,5E,"Alfaro Vive, Carajo (AVC)","Alfaro Lives, Damn It!",AVC,1980,1991,1,11.0,11,...,0,0,0,0,0,0,0,1,0,0
9,11,11K,Ejército de Liberación Nacional (ELN),National Liberation Army,ELN,1963,0,0,51.0,51,...,1,0,0,0,0,0,0,1,0,0
10,12,12L,May 15 Organization for the Liberation of Pale...,May 15 Organization for the Liberation of Pale...,NaN,1979,1984,1,5.0,5,...,0,0,0,1,0,0,1,0,0,1
11,13,13M,Kommando Hau Weg die Scheisse,Get Rid of the Shit Command,NaN,1986,1986,1,0.1,0,...,0,0,0,0,0,0,0,1,0,0
13,15,15O,Communist Party of India-Maoists (CPI-M)/Peopl...,Communist Party of India-Maoists (CPI-M)/Peopl...,CPI-M,2004,0,0,10.0,10,...,0,0,0,0,0,0,0,1,0,0
15,17,17Q,African National Congress (ANC)/Umkhonto we Si...,African National Congress (ANC)/Umkhonto we Si...,ANC,1912,0,0,102.0,33,...,1,0,0,0,1,0,0,0,0,0
17,19,19S,Movimiento 19 de Abril (M-19)/Movimiento 19 de...,April 19th Movement,M-19,1970,1991,1,21.0,20,...,1,0,0,0,0,0,0,1,0,0


In [60]:
original_df[original_df['turnedviolent'] == 1]

,revmodid,revmodcode,organization,english_translation,popular_alternative_name_or_acronym_most_recent,born,died,dead,age,duration,...,participates_in_negotiations,sas,sa,splinter_organization,ethnic,religious_sectarian,national,ideological,apocalyptic,fights_foreign_force
12,14,14N,8-8-88 Uprising/National League for Democracy ...,8-8-88 Uprising/National League for Democracy ...,NLD,1988,0,0,26.0,23,...,0,0,0,0,0,0,0,1,0,0
20,22,22V,Communist Party of Nepal-Maoists (CPN-M)/Unifi...,Communist Party of Nepal-Maoists (CPN-M)/Unifi...,CPM-N,1994,0,0,20.0,10,...,1,0,0,1,0,0,0,1,0,0
50,52,52AZ,Huelga de Brazos Caidos/Comité Nacional de Rec...,Strike of Fallen Arms/National Reconstruction ...,NaN,1944,1944,1,0.9,0,...,1,0,0,0,0,0,0,1,0,0
107,109,109DE,Petubohan Persatuan Pembebasan Pattani/Patani ...,Pattani United Liberation Organization (PULO),PULO,1968,0,0,45.0,45,...,1,0,0,0,1,0,0,0,0,0
170,172,172FP,Andijan rights movement,Andijan rights movement,NaN,2005,2005,1,0.4,0,...,0,0,0,0,0,0,1,0,0,0
191,193,193GK,anti-Suharto movement [bloc: Indonesian Democr...,anti-Suharto movement [bloc: Indonesian Democr...,NaN,1997,1998,1,1.1,1,...,0,0,0,0,0,0,0,1,0,0
223,225,225HQ,Jamaat al-Muslimin/Takfir wal-Hijra/Dinniyeh G...,Repentance and Flight,NaN,1971,0,0,43.0,43,...,0,0,0,0,0,1,0,0,0,1
289,291,291KE,Movimiento 14 de Junio (14J)/1J4,14th of June Movement,14J,1959,1960,1,1.0,1,...,0,0,0,0,0,0,0,1,0,0
309,311,311KY,Bulldozer Revolution [bloc: Democratic Opposit...,Resistance!,NaN,1998,2000,1,2.0,2,...,0,0,0,0,0,0,0,1,0,0
310,312,312KZ,Defiance Campaign,Defiance Campaign,NaN,1952,1961,1,9.0,9,...,0,0,0,1,1,0,0,0,0,0


## Fought in Civil War by Region

In [70]:
latAm = original_df[original_df['latinamerica'] == 1]
# latAm[latAm['foughtincivilwar'] == 1]

In [71]:
africa = original_df[original_df['africa'] == 1]
# africa[africa['foughtincivilwar'] == 1]

In [72]:
midEastnAfrica = original_df[original_df['mideastnafrica'] == 1]
# midEastnAfrica[midEastnAfrica['foughtincivilwar'] == 1]

In [73]:
west = original_df[original_df['west'] == 1]
# west[west['foughtincivilwar'] == 1]

In [74]:
centralEurasia = original_df[original_df['centraleurasia'] == 1]
# centralEurasia[centralEurasia['foughtincivilwar'] == 1]

In [75]:
southEastAsia = original_df[original_df['seasia'] == 1]
# southEastAsia[southEastAsia['foughtincivilwar'] == 1]

In [65]:
eastAsia = original_df[original_df['eastasia'] == 1]
eastAsia[eastAsia['foughtincivilwar'] == 1]

,revmodid,revmodcode,organization,english_translation,popular_alternative_name_or_acronym_most_recent,born,died,dead,age,duration,...,participates_in_negotiations,sas,sa,splinter_organization,ethnic,religious_sectarian,national,ideological,apocalyptic,fights_foreign_force


## Transitioned  to organized crime

In [76]:
latAm = original_df[original_df['latinamerica'] == 1]
latAm[latAm['transitioned_to_organized_crime'] == 1]

,revmodid,revmodcode,organization,english_translation,popular_alternative_name_or_acronym_most_recent,born,died,dead,age,duration,...,participates_in_negotiations,sas,sa,splinter_organization,ethnic,religious_sectarian,national,ideological,apocalyptic,fights_foreign_force
0,2,2B,Sendero Luminoso,Shining Path,NaN,1964,2012,1,48.0,48,...,0,0,0,1,0,0,0,1,0,0
9,11,11K,Ejército de Liberación Nacional (ELN),National Liberation Army,ELN,1963,0,0,51.0,51,...,1,0,0,0,0,0,0,1,0,0
25,27,27AA,Movimiento de Izquierda Revolucionaria (MIR)/J...,Movement of the Revolutionary Left,MIR,1965,0,0,49.0,29,...,0,0,0,1,0,0,0,1,0,0
26,28,28AB,Ejercito Popular de Liberacion (EPL)/Esperanza...,Popular Liberation Army,EPL,1967,0,0,47.0,24,...,1,0,0,1,0,0,0,1,0,0
34,36,36AJ,Alianza Republicana Nacionalista (ARENA)/Ejérc...,National Republican Alliance,ARENA,1976,0,0,38.0,13,...,0,0,0,0,0,0,0,1,0,0
37,39,39AM,Coordinadora Guerrillera Simón Bolívar (CGSB)/...,Simón Bolívar Guerrilla Coordinating Board/Nat...,CGSB,1985,1994,1,9.0,9,...,0,0,0,0,0,0,0,1,0,0
69,71,71BS,Las Autodefensas Unidas de Colombia (AUC),United Self-Defense Forces of Colombia,AUC,1997,2012,1,15.0,15,...,1,0,0,0,0,0,0,1,0,0
302,304,304KR,Fuerza Democrática Nicaragüense (FDN)/Resisten...,Nicaraguan Democratic Force,Contras,1981,1990,1,9.0,9,...,1,0,0,0,0,0,0,1,0,0
371,373,373NI,Hector Riobe Brigade (HRB),Hector Riobe Brigade (HRB),NaN,1982,1984,1,2.0,2,...,0,0,0,0,0,0,0,1,0,0
526,528,528TH,Fuerzas Armadas Revolucionares de Colombia (FA...,Revolutionary Armed Forces of Colombia/Revolut...,FARC-EP,1964,0,0,50.0,50,...,1,0,0,0,0,0,0,1,0,0


In [77]:
africa = original_df[original_df['africa'] == 1]
africa[africa['transitioned_to_organized_crime'] == 1]

,revmodid,revmodcode,organization,english_translation,popular_alternative_name_or_acronym_most_recent,born,died,dead,age,duration,...,participates_in_negotiations,sas,sa,splinter_organization,ethnic,religious_sectarian,national,ideological,apocalyptic,fights_foreign_force
15,17,17Q,African National Congress (ANC)/Umkhonto we Si...,African National Congress (ANC)/Umkhonto we Si...,ANC,1912,0,0,102.0,33,...,1,0,0,0,1,0,0,0,0,0
19,21,21U,União Nacional para a Independêcia Total de An...,National Union for the Total Independence of A...,UNITA,1966,0,0,48.0,36,...,1,0,0,0,0,0,0,1,0,1
38,40,40AN,Lord's Resistance Army (LRA)/Holy Spirit Movem...,Lord's Resistance Army (LRA)/Holy Spirit Movem...,LRA,1986,0,0,28.0,28,...,0,0,0,0,0,0,0,1,1,1
41,43,43AQ,Harakat al-Shabaab al-Mujahedin (HSM),Struggling Youth Movement,al-Shabaab,2004,0,0,10.0,10,...,0,58,1,1,0,1,0,0,0,1
120,122,122DR,Jama'at Ahl as-Sunnah lid-Da'awa wal-Jihad/al-...,Group of the People of Sunnah for the Calling ...,Boko Haram,2004,0,0,10.0,10,...,0,32,1,0,0,1,0,0,1,0
131,133,133EC,Frente para a Libertação do Enclave de Cabinda...,Front for the Liberation of the Enclave of Cab...,FLEC,1963,2011,1,48.0,48,...,1,0,0,0,1,0,0,0,0,1
176,178,178FV,al-Ittihad al-Mahakim al-Islamiyya (IMI),Islamic Courts Union,IMI,2006,2006,1,0.5,0,...,0,9,1,0,0,1,0,0,0,0
317,319,319LG,Pan-Africanist Congress/Pan-Africanist Congres...,Pan-Africanist Congress/Pan-Africanist Congres...,POQO,1959,0,0,55.0,55,...,0,0,0,1,0,0,0,1,0,0
323,325,325LM,Jamaat al-Tawhid wal-Jihad fi Garbi Afriqqiyya...,Movement for Oneness and Jihad in West Africa,NaN,2011,0,0,3.0,3,...,0,1,1,1,0,1,0,0,0,1


In [79]:
midEastnAfrica = original_df[original_df['mideastnafrica'] == 1]
midEastnAfrica[midEastnAfrica['transitioned_to_organized_crime'] == 1]

,revmodid,revmodcode,organization,english_translation,popular_alternative_name_or_acronym_most_recent,born,died,dead,age,duration,...,participates_in_negotiations,sas,sa,splinter_organization,ethnic,religious_sectarian,national,ideological,apocalyptic,fights_foreign_force
8,10,10J,al-Tawhid wal-Jihad/Tanzim Qaidat al-Jihad fi ...,Monotheism and Jihad/al-Qaeda in Iraq (AQI)/Mu...,IS,1999,0,0,15.0,15,...,0,1263,1,0,0,1,0,0,1,1
14,16,16P,Harakat at-Tahrir al-Watani al-Filastini (FATA...,Palestinian National Liberation Movement,Fatah,1958,0,0,56.0,56,...,1,62,1,0,0,0,1,0,0,0
21,23,23W,Hezbollah/Islamic Jihad Organization/Islamic R...,Party of Allah,Hezbollah,1982,0,0,32.0,32,...,1,32,1,0,0,1,0,0,1,0
29,31,31AE,Devrimci Sol/Halk Der,Revolutionary Left,DEV-SOL,1978,1994,1,16.0,16,...,0,0,0,1,0,0,0,1,0,0
32,34,34AH,Group Islamique Armé (GIA),Armed Islamic Group,GIA,1992,2005,1,13.0,13,...,0,3,1,1,0,1,0,0,0,0
44,46,46AT,Munazzamat at-Tahrir al-Filastiniyyeh/Jaish Ta...,Palestine Liberation Organization (PLO)/Palest...,PLO,1964,0,0,50.0,35,...,1,0,0,0,0,0,1,0,0,0
52,54,54BB,Tanzim al-Qaeda fi Bilad al-Maghreb al-Islami/...,al-Qaeda in the Islamic Maghreb (AQIM)/Salafi ...,AQIM,1998,0,0,16.0,16,...,0,37,1,1,0,1,0,0,0,1
61,63,63BK,Mujahedin-e Khalq (MeK),Holy Warriors of the Masses,MeK,1965,0,0,49.0,49,...,0,0,0,1,0,0,0,1,0,0
68,70,70BR,Jaish al-Mahdi (JaM)/Muqawimun/Liwa al-Youm al...,Mahdi Army/Peace Companies,Mahdi Army,2003,0,0,11.0,7,...,1,0,0,0,0,1,0,0,1,1
72,74,74BV,Afwaj al-Muqawama al-Lubnaniyye (AMAL)/Harakat...,Lebanese Resistance Detachments,Amal,1974,0,0,40.0,40,...,0,3,1,0,0,1,0,0,0,1


In [80]:
west = original_df[original_df['west'] == 1]
west[west['transitioned_to_organized_crime'] == 1]

,revmodid,revmodcode,organization,english_translation,popular_alternative_name_or_acronym_most_recent,born,died,dead,age,duration,...,participates_in_negotiations,sas,sa,splinter_organization,ethnic,religious_sectarian,national,ideological,apocalyptic,fights_foreign_force
2,4,4D,Óghlaigh na hÉireann/Cumann na mBan/Fianna na ...,Provisional Irish Republican Army (PIRA)/Irish...,PIRA,1969,2005,1,36.0,28,...,1,0,0,0,0,1,0,0,0,0
16,18,18R,Fronte di Liberazione Naziunale Corsu (FLNC)/C...,Corsican National Liberation Front,FLNC,1976,2014,1,38.0,38,...,1,0,0,0,1,0,0,0,0,0
27,29,29AC,Ulster Volunteer Force (UVF),Ulster Volunteer Force (UVF),UVF,1966,0,0,48.0,44,...,0,0,0,0,0,1,0,0,0,0
36,38,38AL,Grupos de Resistencia Antifascista Primero de ...,First of October Antifascist Resistance Group/...,GRAPO,1975,2007,1,32.0,32,...,0,0,0,0,0,0,0,1,0,1
54,56,56BD,Rote Armee Fraktion (RAF)/Baader-Meinhof Group...,Red Army Faction,RAF,1968,1998,1,30.0,30,...,0,0,0,0,0,0,0,1,0,1
55,57,57BE,Páirtí Poblachtach Sóisialach na hÉireann/Arm ...,Irish Republican Socialist Party (IRSP)/Irish ...,INLA,1974,0,0,40.0,40,...,0,0,0,1,0,0,0,1,0,0
57,59,59BG,Epanastatiki Organosi Dekaefta Noemvri/Social ...,Revolutionary Organization November 17 (17N),17N,1975,2002,1,27.0,27,...,0,0,0,0,0,0,0,1,0,1
86,88,88CJ,Ushria Çlirimatare Kosovës,Kosovo Liberation Army (KLA),KLA,1992,1999,1,7.0,7,...,0,0,0,0,0,1,0,0,0,0
99,101,101CW,Óghlaigh na hÉireann/Republican Action Against...,Real Irish Republican Army (RIRA),RIRA,1997,0,0,17.0,17,...,0,0,0,1,0,1,0,0,0,0
122,124,124DT,Óglaigh na hÉireann/Sinn Féin Poblachtach/Cuma...,Continuity Irish Republican Army (CIRA)/The Ir...,CIRA,1986,0,0,28.0,28,...,0,0,0,1,0,1,0,0,0,0


In [82]:
centralEurasia = original_df[original_df['centraleurasia'] == 1]
centralEurasia[centralEurasia['transitioned_to_organized_crime'] == 1]

,revmodid,revmodcode,organization,english_translation,popular_alternative_name_or_acronym_most_recent,born,died,dead,age,duration,...,participates_in_negotiations,sas,sa,splinter_organization,ethnic,religious_sectarian,national,ideological,apocalyptic,fights_foreign_force
82,84,84CF,Riyad as-Saliheyn Martyrs' Brigade (RSMB),Riyad as-Saliheyn Martyrs' Brigade (RSMB),RSMB,2002,2009,1,7.0,7,...,0,46,1,0,0,1,0,0,0,0
83,85,85CG,Imarat Kavkaz,Caucasus Emirate,NaN,2007,0,0,7.0,7,...,0,56,1,0,0,1,0,0,1,0
152,154,154EX,Oʻzbekiston Islomiy Harakati,Islamic Movement of Uzbekistan (IMU),IMU,1998,0,0,16.0,16,...,0,14,1,1,0,1,0,0,0,1
232,234,234HZ,Dagestan Liberation Army,Dagestan Liberation Army,NaN,1999,2002,1,3.0,3,...,0,0,0,0,1,0,0,0,0,0
241,243,243II,Taliban/Da Afghanistan Islami Amarat,Islamic Emirate of Afghanistan,Taliban,1991,0,0,23.0,23,...,0,755,1,0,0,1,0,0,0,1
244,246,246IL,Jund al-Khalifa,Soldiers of the Caliphate,NaN,2011,2012,1,1.0,1,...,0,3,1,0,0,1,0,0,0,0


In [83]:
southEastAsia = original_df[original_df['seasia'] == 1]
southEastAsia[southEastAsia['transitioned_to_organized_crime'] == 1]

,revmodid,revmodcode,organization,english_translation,popular_alternative_name_or_acronym_most_recent,born,died,dead,age,duration,...,participates_in_negotiations,sas,sa,splinter_organization,ethnic,religious_sectarian,national,ideological,apocalyptic,fights_foreign_force
42,44,44AR,Moro National Liberation Front (MNLF)/Bangsamo...,Moro National Liberation Front (MNLF)/Bangsamo...,MNLF,1969,0,0,45.0,45,...,1,0,0,0,1,0,0,0,0,0
43,45,45AS,Abu Sayyaf Group,Abu Sayyaf Group,Abu Sayyaf,1991,0,0,23.0,23,...,0,0,0,1,0,1,0,0,0,0
46,48,48AV,Muhajir Qami Movement (MQM)/Haqiqi Muhajir Qam...,Muhajir Qami Movement (MQM)/Haqiqi Muhajir Qam...,MQM,1984,0,0,30.0,20,...,0,0,0,0,1,0,0,0,0,0
53,55,55BC,Lashkar-e Toiba (LeT)/al-Mansoorian/Jamaat ud-...,Army of the Pure,LeT,1985,0,0,29.0,29,...,0,13,1,0,0,1,0,0,0,0
60,62,62BJ,Gerakan Aceh Merdeka (GAM)/Aceh Sumatra Nation...,Free Aceh Movement,GAM,1976,2005,1,29.0,29,...,1,0,0,0,1,0,0,0,0,0
65,67,67BO,Hizb-ul Mujahedin,Party of Holy Warriors,HM,1989,0,0,25.0,25,...,1,2,1,0,0,1,0,0,0,0
71,73,73BU,Jema'a Islamiyya (JI),Islamic Group,JI,1993,0,0,21.0,21,...,0,11,1,0,0,1,0,0,0,1
144,146,146EP,Organisasi Papua Merdeka (OPM)/West Papua Nati...,Free Papua Movement,OPM,1963,0,0,51.0,51,...,1,0,0,0,1,0,0,0,0,0
267,269,269JI,Jema'ah Ashorut Tauhid,Group of the Partisans of Monotheism,JAT,2008,0,0,6.0,6,...,0,2,1,1,0,1,0,0,0,0
337,339,339MA,Burma Communist Party (BCP)/Burma Communist Pa...,Burma Communist Party (BCP)/Burma Communist Pa...,BCP,1939,0,0,75.0,75,...,0,0,0,0,0,0,0,1,0,0


In [84]:
eastAsia = original_df[original_df['eastasia'] == 1]
eastAsia[eastAsia['transitioned_to_organized_crime'] == 1]

,revmodid,revmodcode,organization,english_translation,popular_alternative_name_or_acronym_most_recent,born,died,dead,age,duration,...,participates_in_negotiations,sas,sa,splinter_organization,ethnic,religious_sectarian,national,ideological,apocalyptic,fights_foreign_force
269,271,271JK,Turkestan Islamic Party/East Turkestan Islamic...,Turkestan Islamic Party/East Turkestan Islamic...,ETIM,1997,0,0,17.0,17,...,0,2,1,0,0,1,0,0,0,1
276,278,278JR,Aleph/Aum Shinrikyo/Aum Shinsen no Kai (ASNK),Supreme Truth,NaN,1984,0,0,30.0,30,...,0,0,0,0,0,0,0,1,1,0
